In [1]:
import os
import sys
from pathlib import Path
import gc
import ee
import geemap
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns 
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling
from dataclasses import dataclass, field
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Polygon, MultiPolygon
from matplotlib.path import Path as MplPath
import warnings
import unicodedata
import re

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
warnings.filterwarnings("ignore")

@dataclass
class MeritConfig:
    # Caminhos relativos
    BASE_DIR: Path = Path("..")
    PATH_MUNICIPIOS: Path = BASE_DIR / "inputs" / "BR_Municipios_2024" / "BR_Municipios_2024.shp"
    PATH_ESTADOS: Path = BASE_DIR / "inputs" / "BR_UF_2024" / "BR_UF_2024.shp"
    PATH_OUTPUT: Path = BASE_DIR / "output"
    
    NOME_MUNICIPIO: str = 'Sobradinho'
    SIGLA_UF: str = 'BA'
    ASSET_ID: str = "MERIT/Hydro/v1_0_1"
    
    RIVER_THRESHOLD_KM2: int = 5 
    SCALE_METERS: int = 90
    CRS_VISUAL: str = 'EPSG:3857'
    CRS_LATLON: str = 'EPSG:4674'

class GeoUtils:
    @staticmethod
    def normalize_string(text: str) -> str:
        if not isinstance(text, str): return ""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower().strip()

    @staticmethod
    def sanitize_filename(nome: str) -> str:
        return re.sub(r'[^\w\s-]', '', GeoUtils.normalize_string(nome)).strip().replace(' ', '_')

class MeritService:
    def __init__(self, config: MeritConfig):
        self.cfg = config
        self._authenticate()

    def _authenticate(self):
        try:
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        except:
            ee.Authenticate()
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

    def process_data(self, region_ee):
        merit = ee.Image(self.cfg.ASSET_ID)
        elev = merit.select('elv')
        upa = merit.select('upa')
        hand = merit.select('hnd')
        
        slope = ee.Terrain.slope(elev)
        river_mask = upa.gt(self.cfg.RIVER_THRESHOLD_KM2)
        width = merit.select('wth').updateMask(river_mask)
        
        export_img = elev.unmask(-9999).rename('elevation') \
            .addBands(river_mask.unmask(0).rename('river_network')) \
            .addBands(hand.unmask(-9999).rename('hand')) \
            .addBands(width.unmask(0).rename('width')) \
            .addBands(slope.unmask(-9999).rename('slope'))
            
        return export_img.clip(region_ee.geometry())

    def download_raster(self, image: ee.Image, region_ee, output_path: Path):
        if output_path.exists():
            try: os.remove(output_path)
            except: pass
            
        geemap.download_ee_image(
            image,
            filename=str(output_path),
            region=region_ee.geometry(),
            scale=self.cfg.SCALE_METERS,
            crs=self.cfg.CRS_VISUAL,
            overwrite=True,
            num_threads=4 
        )

class MeritRenderer:
    def __init__(self, config: MeritConfig):
        self.cfg = config

    def _styled_box(self, ax, title):
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
            spine.set_edgecolor('#333333')
            spine.set_linewidth(1)
        ax.set_title(title, fontsize=9, weight='bold', pad=6, backgroundcolor='#e0e0e0', color='black')

    def _setup_grid_lines(self, ax, raster_bounds):
        minx, miny, maxx, maxy = raster_bounds
        x_target = np.linspace(minx, maxx, 4)
        y_target = np.linspace(miny, maxy, 4)
        
        pts_metric = gpd.points_from_xy(x_target, y_target, crs=self.cfg.CRS_VISUAL)
        pts_latlon = pts_metric.to_crs(self.cfg.CRS_LATLON)
        
        ax.set_xticks(x_target)
        ax.set_yticks(y_target)
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{pts_latlon[p].x:.2f}°" if p < len(pts_latlon) else ""))
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, p: f"{pts_latlon[p].y:.2f}°" if p < len(pts_latlon) else ""))
        ax.tick_params(axis='both', which='major', labelsize=7, direction='in', pad=4)
        ax.grid(True, linestyle='--', alpha=0.3, color='#444444', linewidth=0.5, zorder=3)

    def _draw_minimap(self, ax, mun_gdf):
        self._styled_box(ax, f"LOCALIZAÇÃO - {self.cfg.SIGLA_UF}")
        try:
            if self.cfg.PATH_ESTADOS.exists():
                states = gpd.read_file(self.cfg.PATH_ESTADOS)
                state = states[states['SIGLA_UF'] == self.cfg.SIGLA_UF.upper()]
                if not state.empty:
                    state.to_crs(self.cfg.CRS_VISUAL).plot(ax=ax, facecolor='#f4f4f4', edgecolor='#555555', linewidth=0.5)
                    mun_gdf.to_crs(self.cfg.CRS_VISUAL).plot(ax=ax, facecolor='#d62728', edgecolor='none')
            else:
                ax.text(0.5, 0.5, "Shape UF N/A", ha='center', fontsize=7)
        except: pass

    def _draw_metadata(self, ax):
        self._styled_box(ax, "")
        ax.text(0.5, 0.95, "METADADOS", ha='center', va='top', fontsize=9, weight='bold', transform=ax.transAxes)
        txt = (
            f"Município: {self.cfg.NOME_MUNICIPIO}\n"
            f"Dataset: MERIT Hydro v1.0.1\n"
            f"Resolução: {self.cfg.SCALE_METERS}m\n"
            f"Processamento: GEE & Python"
        )
        ax.text(0.5, 0.75, txt, ha='center', va='top', fontsize=8, linespacing=1.6, transform=ax.transAxes)

    def _sanitize_data(self, data_array):
        clean = data_array.copy()
        clean[np.isinf(clean)] = np.nan
        clean[clean <= -9000] = np.nan
        return clean[~np.isnan(clean)]

    def _draw_quick_stats(self, ax, data_array, unit):
        self._styled_box(ax, "RESUMO")
        valid = self._sanitize_data(data_array)
        if valid.size == 0:
            ax.text(0.5, 0.5, "Sem dados válidos", ha='center', fontsize=8)
            return
        mn, mx, me = np.min(valid), np.max(valid), np.mean(valid)
        std = np.std(valid)
        txt = (
            f"Mínimo: {mn:.1f} {unit}\n"
            f"Máximo: {mx:.1f} {unit}\n"
            f"Média:  {me:.1f} {unit}\n"
            f"Desvio: {std:.1f}"
        )
        ax.text(0.5, 0.5, txt, fontsize=8, ha='center', va='center', fontfamily='monospace', linespacing=1.6)

    def generate_map(self, raster_path, aoi_gdf, map_type='elevation'):
        fig = plt.figure(figsize=(11.69, 8.27))
        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.1, figure=fig)
        ax_map = fig.add_subplot(gs[0])
        
        configs = {
            'elevation': (1, 'terrain', "ALTIMETRIA DIGITAL", "(m)", None, None),
            'slope': (5, 'magma_r', "DECLIVIDADE (RELEVO)", "Graus (°)", 0, 30),
            'hand': (3, 'Blues_r', "MODELO HAND (RISCO HÍDRICO)", "(m)", 0, 20),
            'network': (4, 'GnBu', "REDE DE DRENAGEM ESTIMADA", "(m)", None, None)
        }
        band_idx, cmap, title, unit, vmin, vmax = configs[map_type]

        with rasterio.open(raster_path) as src:
            overscale = max(1, src.width // 2000)
            data = src.read(band_idx, out_shape=(src.height // overscale, src.width // overscale), resampling=Resampling.nearest)
            data = data.astype(float)
            data[data <= -9000] = np.nan
            if map_type == 'network': data[data <= 0] = np.nan
            
            extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
            im = ax_map.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent, zorder=1)
            self._setup_grid_lines(ax_map, src.bounds)

        aoi_proj = aoi_gdf.to_crs(self.cfg.CRS_VISUAL)
        aoi_proj.plot(ax=ax_map, facecolor='none', edgecolor='black', linewidth=1.5, zorder=2)
        ax_map.add_artist(ScaleBar(1, units='m', location='lower left', box_alpha=0.7, font_properties={'family': 'serif', 'size': 8}))
        
        ax_map.annotate('N', xy=(0.95, 0.95), xytext=(0.95, 0.88), 
                        arrowprops=dict(facecolor='black', width=4, headwidth=10),
                        ha='center', va='center', fontsize=12, weight='bold', xycoords='axes fraction', zorder=10)
        ax_map.set_title(f"{title}\n{self.cfg.NOME_MUNICIPIO} - {self.cfg.SIGLA_UF}", fontsize=14, weight='bold', pad=15)

        gs_side = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[1], height_ratios=[0.25, 0.1, 0.25, 0.4], hspace=0.3)
        self._draw_minimap(fig.add_subplot(gs_side[0]), aoi_gdf)
        
        ax_leg = fig.add_subplot(gs_side[1])
        self._styled_box(ax_leg, "LEGENDA")
        cbar = plt.colorbar(im, cax=ax_leg, orientation='horizontal', fraction=0.5, pad=0.1)
        cbar.set_label(unit, fontsize=8, weight='bold')

        self._draw_metadata(fig.add_subplot(gs_side[2]))
        self._draw_quick_stats(fig.add_subplot(gs_side[3]), data, unit.split(' ')[0])
        return fig

    def generate_dashboard(self, raster_path):
        fig = plt.figure(figsize=(11.69, 8.27))
        fig.suptitle(f"ANÁLISE MORFOMÉTRICA - {self.cfg.NOME_MUNICIPIO}", fontsize=16, weight='bold', y=0.95)
        gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], hspace=0.3, wspace=0.3)
        
        with rasterio.open(raster_path) as src:
            scale = max(1, src.width // 1000)
            elev = self._sanitize_data(src.read(1, out_shape=(src.height//scale, src.width//scale), resampling=Resampling.bilinear))
            hand = self._sanitize_data(src.read(3, out_shape=(src.height//scale, src.width//scale), resampling=Resampling.nearest))
            slope = self._sanitize_data(src.read(5, out_shape=(src.height//scale, src.width//scale), resampling=Resampling.bilinear))

        if elev.size == 0: return fig

        ax_hyp = fig.add_subplot(gs[0, 0])
        sorted_elev = np.sort(elev)
        ax_hyp.plot(sorted_elev, np.linspace(0, 100, len(sorted_elev)), color='brown', linewidth=2)
        ax_hyp.set_title("Curva Hipsométrica", weight='bold', fontsize=10)
        ax_hyp.grid(True, linestyle='--', alpha=0.5)

        ax_slope = fig.add_subplot(gs[0, 1])
        sns.histplot(slope, kde=True, ax=ax_slope, color='purple', bins=30, stat="percent")
        ax_slope.set_title("Distribuição de Declividade", weight='bold', fontsize=10)
        ax_slope.set_xlim(0, 45)

        ax_risk = fig.add_subplot(gs[1, 0])
        total = len(hand)
        if total > 0:
            sizes = [np.sum(hand < 5), np.sum((hand >= 5) & (hand < 15)), np.sum(hand >= 15)]
            colors = ['#ff6666', '#ffcc00', '#99ff99']
            ax_risk.pie(sizes, labels=['Alto Risco', 'Médio', 'Baixo'], colors=colors, startangle=90)
        ax_risk.set_title("Risco de Inundação (HAND)", weight='bold', fontsize=10)

        ax_box = fig.add_subplot(gs[1, 1])
        ax_box.boxplot([elev, slope*10], labels=['Altitude', 'Declividade x10'], patch_artist=True)
        ax_box.set_title("Dispersão", weight='bold', fontsize=10)
        ax_box.grid(True, axis='y', linestyle='--', alpha=0.5)
        
        return fig

class MeritPipeline:
    def __init__(self, config: MeritConfig):
        self.cfg = config
        self.service = MeritService(config)
        self.renderer = MeritRenderer(config)

    def run(self):
        temp_tif = None
        try:
            print(f"--- INICIANDO MERIT: {self.cfg.NOME_MUNICIPIO} ---")
            
            full_gdf = gpd.read_file(self.cfg.PATH_MUNICIPIOS)
            full_gdf['norm_nm'] = full_gdf['NM_MUN'].apply(GeoUtils.normalize_string)
            aoi = full_gdf[(full_gdf['norm_nm'] == GeoUtils.normalize_string(self.cfg.NOME_MUNICIPIO)) & 
                           (full_gdf['SIGLA_UF'] == self.cfg.SIGLA_UF.upper())].dissolve()
            
            if aoi.empty: raise ValueError("Município não encontrado.")
            
            print("1. Processando MERIT Hydro + Declividade...")
            aoi_ee = geemap.geopandas_to_ee(aoi.to_crs("EPSG:4326"))
            img_hydro = self.service.process_data(aoi_ee)
            
            safe_name = GeoUtils.sanitize_filename(self.cfg.NOME_MUNICIPIO)
            self.cfg.PATH_OUTPUT.mkdir(parents=True, exist_ok=True)
            temp_tif = self.cfg.PATH_OUTPUT / f"TEMP_MERIT_{safe_name}.tif"
            
            print("2. Baixando Raster...")
            self.service.download_raster(img_hydro, aoi_ee, temp_tif)
            
            if not temp_tif.exists(): raise FileNotFoundError("Erro no download.")
            
            print("3. Gerando Relatório Avançado...")
            figs = [
                self.renderer.generate_map(temp_tif, aoi, 'elevation'),
                self.renderer.generate_map(temp_tif, aoi, 'slope'),     
                self.renderer.generate_map(temp_tif, aoi, 'hand'),
                self.renderer.generate_map(temp_tif, aoi, 'network'),
                self.renderer.generate_dashboard(temp_tif)              
            ]
            
            out_pdf = self.cfg.PATH_OUTPUT / f"RELATORIO_TOPOGRAFIA_{safe_name}.pdf"
            with PdfPages(out_pdf) as pdf:
                for f in figs:
                    pdf.savefig(f, bbox_inches='tight', dpi=300)
                    plt.close(f)
            
            print(f"Relatório Salvo: {out_pdf}")
            
        except Exception as e:
            print(f"ERRO CRÍTICO: {e}")
            import traceback; traceback.print_exc()
        finally:
            if temp_tif and temp_tif.exists():
                try: 
                    gc.collect() 
                    temp_tif.unlink()
                except: pass

if __name__ == "__main__":
    cfg = MeritConfig(NOME_MUNICIPIO='Três Corações', SIGLA_UF='MG')
    MeritPipeline(cfg).run()

c:\Users\pedro\anaconda3\envs\hidrokit\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.10) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


--- INICIANDO MERIT: Três Corações ---
1. Processando MERIT Hydro + Declividade...
2. Baixando Raster...
ERRO CRÍTICO: Please install geedim using `pip install geedim` or `conda install -c conda-forge geedim`


Traceback (most recent call last):
  File "c:\Users\pedro\anaconda3\envs\hidrokit\lib\site-packages\geemap\common.py", line 12490, in download_ee_image
    import geedim as gd
ModuleNotFoundError: No module named 'geedim'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\pedro\AppData\Local\Temp\ipykernel_20704\2516585624.py", line 286, in run
    self.service.download_raster(img_hydro, aoi_ee, temp_tif)
  File "C:\Users\pedro\AppData\Local\Temp\ipykernel_20704\2516585624.py", line 92, in download_raster
    geemap.download_ee_image(
  File "c:\Users\pedro\anaconda3\envs\hidrokit\lib\site-packages\geemap\common.py", line 12492, in download_ee_image
    raise ImportError(
ImportError: Please install geedim using `pip install geedim` or `conda install -c conda-forge geedim`
